In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import pystan, pickle, popmachine, scipy
import pandas as pd

from pystan_cache.pystan_cache import caching_stan

In [ ]:
pystan.__version__

In [ ]:
machine = popmachine.Machine('sqlite:///../popmachine_local/.popmachine.db')

In [ ]:
gp_multi = caching_stan.stan_model(file='stan-models/gp_multi.stan')
gp_multi_marginal = caching_stan.stan_model(file='stan-models/gp_multi_marginal.stan')
# gp_multi_marginal_gamma = caching_stan.stan_model(file='stan-models/gp_multi_marginal_gamma.stan')

In [ ]:
plates = ['20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
       u'20150715 PQ 8', u'20150702 PQ 6', u'20150607 PQ 4',
       u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9']

In [ ]:
np.kron?

In [ ]:
dm.shape

In [ ]:
temp = np.zeros((dm.shape[0], 2))
temp[:, 0] = 1
temp[::2, 1] = 1

r = np.zeros((dm.shape[0], 3))
r[:5, 0] = 1
r[5:10,1] = 1
r[10:, 2] = 1

# plt.imshow(np.kron(temp[0,:], r[0,:]), aspect='auto')

In [ ]:
temp.shape, r.shape

In [ ]:
temp

In [ ]:
r

In [ ]:
np.array([np.kron(temp[i,:], r[i,:]) for i in range(dm.shape[0])])

In [ ]:
for plate in plates:
    
    ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    
    ymean, ystd = y.mean(), y.std()
    y = (y-y.mean())/y.std()
    x = (xraw-xraw.mean())/xraw.std()
    
    dm = np.ones((y.shape[1], 1))
    
    p = dm.shape[1]
    n = x.shape[0]

    train_data = {
        'N': n,
        'P':y.shape[1],
        'K':dm.shape[1],
        'L':1,
        'prior':[1],     
        'design': dm
    }

    train_data['y'] = y.T
    train_data['x'] = x[:,0]

    train_data['alpha_prior'] = [[1,1]]
    train_data['length_scale_prior'] = [[.5, .5]]
    train_data['marginal_alpha_prior'] = [.5, .1]
    train_data['marginal_lengthscale_prior'] = [8, 2.0]

    train_data['sigma_prior'] = [.1,1.5]

    tsamples = gp_multi_marginal.sampling(data=train_data, chains=4, iter=2000, control = {'adapt_delta': 0.8})
    
    summary = tsamples.summary()
    summary = pd.DataFrame(summary['summary'], columns=summary['summary_colnames'], index=summary['summary_rownames'])
    
    tsamp = tsamples.extract(permuted=True)
    pickle.dump(tsamp, open('samples/ura3-0.0mMPQ-replicate-%s-samples.pkl'%plate, 'wb'))
    
    print plate
    print summary.Rhat.describe()

In [ ]:
for plate in plates:
    
    ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    
    ymean, ystd = y.mean(), y.std()
    y = (y-y.mean())/y.std()
    x = (xraw-xraw.mean())/xraw.std()
    
    dm = np.ones((y.shape[1], 1))
    
    p = dm.shape[1]
    n = x.shape[0]

    train_data = {
        'N': n,
        'P':y.shape[1],
        'K':dm.shape[1],
        'L':1,
        'prior':[1],     
        'design': dm
    }

    train_data['y'] = y.T
    train_data['x'] = x[:,0]

    train_data['alpha_prior'] = [[1,1]]
    train_data['length_scale_prior'] = [[.5, .5]]
    train_data['marginal_alpha_prior'] = [.5, .1]
    train_data['marginal_lengthscale_prior'] = [8, 2.0]

    train_data['sigma_prior'] = [.1,1.5]

    tsamples = gp_multi.sampling(data=train_data, chains=4, iter=2000, control = {'adapt_delta': 0.8})
    
    summary = tsamples.summary()
    summary = pd.DataFrame(summary['summary'], columns=summary['summary_colnames'], index=summary['summary_rownames'])
    
    tsamp = tsamples.extract(permuted=True)
    pickle.dump(tsamp, open('samples/ura3-0.0mMPQ-replicate-mnull-%s-samples.pkl'%plate, 'wb'))
    
    print plate
    print summary.Rhat.describe()

In [ ]:
plates

In [ ]:
plate = plates[-2]

In [ ]:
tsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-%s-samples.pkl'%plate, 'rb'))
nsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-mnull-%s-samples.pkl'%plate, 'rb'))

In [ ]:
plt.figure(figsize=(len(plates) * 4, 4))

for p, plate in enumerate(plates):
    
    ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    
    tsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-%s-samples.pkl'%plate, 'rb'))
    nsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-mnull-%s-samples.pkl'%plate, 'rb'))

    plt.subplot(1, len(plates), p+1)
    i = 0

    plt.plot(xraw, nsamp['f'][:,i,:].mean(0),)
    plt.fill_between(xraw[:,0], 
                     nsamp['f'][:,i,:].mean(0)-2*nsamp['f'][:,i,:].std(0),
                     nsamp['f'][:,i,:].mean(0)+2*nsamp['f'][:,i,:].std(0),alpha=.1)

    plt.plot(xraw, tsamp['f'][:,i,:].mean(0),)
    plt.fill_between(xraw[:,0], 
                     tsamp['f'][:,i,:].mean(0)-2*tsamp['f'][:,i,:].std(0),
                     tsamp['f'][:,i,:].mean(0)+2*tsamp['f'][:,i,:].std(0),alpha=.3)

In [ ]:
plt.figure(figsize=(8, 4))

for p, plate in enumerate(plates):
    
    ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    ymean, ystd = y.mean(), y.std()
    
    xraw = xraw[:31,:]
    
    tsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-%s-samples.pkl'%plate, 'rb'))
    nsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-mnull-%s-samples.pkl'%plate, 'rb'))

    plt.subplot(121)    
    temp = nsamp['f'][:,0,:31]
    temp = temp*ystd + ymean
    plt.plot(xraw, temp.mean(0),)
    plt.fill_between(xraw[:,0], 
                     temp.mean(0)-2*temp.std(0),
                     temp.mean(0)+2*temp.std(0),alpha=.1)

    plt.subplot(122)    
    temp = tsamp['f'][:,0,:31]
    temp = temp*ystd + ymean
    plt.plot(xraw, temp.mean(0),)
    plt.fill_between(xraw[:,0], 
                     temp.mean(0)-2*temp.std(0),
                     temp.mean(0)+2*temp.std(0),alpha=.1)
    

plt.subplot(121)
plt.xlabel('time (h)', fontsize=16)
plt.ylabel('log(OD)', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.subplot(122)
plt.xlabel('time (h)', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()

plt.savefig('figures/ura3_0.0mMPQ-replicate-functions.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(8, 4))

for p, plate in enumerate(plates):
    
    ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    ymean, ystd = y.mean(), y.std()
    
    xraw = xraw[:31,:]
    
    tsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-%s-samples.pkl'%plate, 'rb'))
    nsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-mnull-%s-samples.pkl'%plate, 'rb'))
    
    temp = nsamp['f'][:,0,:31]
    temp = temp*ystd + ymean
    
    temp2 = tsamp['f'][:,0,:31]
    temp2 = temp2*ystd + ymean
    
    plt.boxplot([np.log10(temp2.var(0)/temp.var(0))], positions=[p], showfliers=False)

plt.xticks([])
plt.plot([-1, len(plates)], [0,0], c='k')   
plt.xlim(-1, len(plates))
plt.yticks(fontsize=16)

plt.savefig('figures/ura3_0.0mMPQ-replicate-functionInterval.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(16, 4))

for p, plate in enumerate(plates):
    
    ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    ymean, ystd = y.mean(), y.std()
    
    xraw = xraw[:31,:]
    
    tsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-%s-samples.pkl'%plate, 'rb'))
    nsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-mnull-%s-samples.pkl'%plate, 'rb'))
    
    for r, param in enumerate(['alpha', 'length_scale', 'marginal_alpha', 'marginal_lengthscale']):
        
        temp = tsamp[param]
        z = np.linspace(temp.min()*.7, temp.max()*1.3)
        kde = scipy.stats.gaussian_kde(temp)
        
        plt.subplot(1, 4, r+1)
        plt.plot(z, kde(z))

In [ ]:
plt.figure(figsize=(12, 4))

for p, plate in enumerate(plates):
    
    ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    ymean, ystd = y.mean(), y.std()
    
    xraw = xraw[:31,:]
    
    tsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-%s-samples.pkl'%plate, 'rb'))
    nsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-mnull-%s-samples.pkl'%plate, 'rb'))
    
    for r, param in enumerate(['alpha', 'length_scale', 'sigma']):
        
        temp = tsamp[param]
        temp2 = nsamp[param]
        
        plt.subplot(1,3,r+1)
        plt.boxplot([temp2], positions=[p*2], showfliers=False)
        plt.boxplot([temp], positions=[p*2 + 1], showfliers=False, boxprops={'color':'blue'})
        
plt.subplot(131)
plt.xlim(-1, 2*len(plates))

plt.subplot(132)
plt.xlim(-1, 2*len(plates))

plt.subplot(133)
plt.xlim(-1, 2*len(plates))

In [ ]:
plt.figure(figsize=(9,9))

for p, plate in enumerate(plates):
    
    ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    ymean, ystd = y.mean(), y.std()
    
    xraw = xraw[:31,:]
    
    tsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-%s-samples.pkl'%plate, 'rb'))
    nsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-mnull-%s-samples.pkl'%plate, 'rb'))
    
        
    #temp = tsamp['alpha'] + tsamp['sigma'] + tsamp['marginal_alpha']
    #temp2 = nsamp['alpha'] + nsamp['sigma']
    
    temp = tsamp['sigma'] + tsamp['marginal_alpha']
    temp2 = nsamp['sigma']

#     buff = 5
#     plt.boxplot([temp2], positions=[p*buff], showfliers=False, vert=True, widths=[.5])
#     plt.boxplot([tsamp['sigma']], positions=[p*buff + 1], showfliers=False, boxprops={'color':'blue'}, widths=[.5], vert=True)
#     plt.boxplot([tsamp['marginal_alpha']], positions=[p*buff + 2], showfliers=False, boxprops={'color':'red'}, widths=[.5], vert=True)
#     plt.boxplot([tsamp['sigma'] + tsamp['marginal_alpha']], positions=[p*buff + 3], showfliers=False, boxprops={'color':'green'}, widths=[.5], vert=True)

    plt.subplot(3, 3, p+1)
    plt.boxplot([temp2], positions=[0], showfliers=False, vert=False, widths=[.5])
    plt.boxplot([tsamp['sigma']], positions=[1], showfliers=False, boxprops={'color':'blue'}, widths=[.5], vert=False)
    plt.boxplot([tsamp['marginal_alpha']], positions=[2], showfliers=False, boxprops={'color':'red'}, widths=[.5], vert=False)
    plt.boxplot([tsamp['sigma'] + tsamp['marginal_alpha']], positions=[3], showfliers=False, boxprops={'color':'green'}, widths=[.5], vert=False)
    plt.ylim(-1, 4)
    
    if p % 3 == 0:
        plt.yticks(range(4), ['']*4)
    else:
        plt.yticks([])
        
    plt.xticks(fontsize=12)
    
plt.tight_layout()
plt.savefig('figures/ura3_0.0mMPQ-varianceTotal-replicate.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12, 4))

for p, plate in enumerate(plates):
    
    ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    ymean, ystd = y.mean(), y.std()
    
    xraw = xraw[:31,:]
    
    tsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-%s-samples.pkl'%plate, 'rb'))
    nsamp = pickle.load(open('samples/ura3-0.0mMPQ-replicate-mnull-%s-samples.pkl'%plate, 'rb'))
    
        
    #temp = tsamp['alpha'] + tsamp['sigma'] + tsamp['marginal_alpha']
    #temp2 = nsamp['alpha'] + nsamp['sigma']
    
    temp = tsamp['sigma']
    temp2 = nsamp['sigma']

    plt.boxplot([temp2], positions=[p*3], showfliers=False, vert=True, widths=[.5])
    plt.boxplot([temp], positions=[p*3 + 1], showfliers=False, boxprops={'color':'blue'}, widths=[.5], vert=True)
        
plt.xlim(-1, 3*len(plates)-1)
plt.xticks([])

plt.savefig('figures/ura3_0.0mMPQ-varianceNoise-replicate.pdf', bbox_inches='tight')

In [ ]:
line = line[0]

In [ ]:
line.properties()

# null model 

In [ ]:
null_train_data = train_data.copy()
null_train_data['design'] = train_data['design'][:,:2]
null_train_data['prior'] = null_train_data['prior'][:2]
null_train_data['length_scale_prior'] = null_train_data['length_scale_prior'][:2]
null_train_data['alpha_prior'] = null_train_data['alpha_prior'][:2]
null_train_data['K'] = null_train_data['L'] = 2

In [ ]:
nullSamples = gp_multi.sampling(data=null_train_data, chains=4, iter=2000, control = {'adapt_delta': 0.8})

In [ ]:
nullSamples

In [ ]:
plt.figure(figsize=(10,4))
nullSamples.traceplot(['length_scale', 'alpha', 'sigma','lp__'])
plt.tight_layout()

In [ ]:
nsamp = nullSamples.extract(permuted=True)

In [ ]:
ncol = 2
nrow = int(1.*(dm.shape[1]-1)/ncol) + 1

plt.figure(figsize=(4*ncol, 4*nrow))

for i in range(2):
    
    plt.subplot(dm.shape[1]/ncol + 1, ncol, i + 1)
    
    plt.plot(x, nsamp['f'][:,i,:].mean(0),)
    #plt.plot(x, tsamp['f'][:,i,:].T,c='k', alpha=.2)
    plt.fill_between(x[:,0], 
                     nsamp['f'][:,i,:].mean(0)-2*nsamp['f'][:,i,:].std(0),
                     nsamp['f'][:,i,:].mean(0)+2*nsamp['f'][:,i,:].std(0),alpha=.1)
    plt.plot([x.min(), x.max()], [0, 0], lw=3, c='k')
    
    if i > 1:
        plt.ylim(-.48, .48)
    
    #plt.plot(x[train_ind],f[i,train_ind].T,'--')

In [ ]:
for i in range(2):

    temp = nsamp['alpha'][:,i]

    z = np.linspace(temp.min()*.7, temp.max()*1.3)
    kde = scipy.stats.gaussian_kde(temp)


    plt.plot(z, kde(z), label='alpha %d'%i)

plt.semilogx()    
plt.legend()

In [ ]:
for i in range(2):

    temp = nsamp['length_scale'][:,i]

    z = np.linspace(temp.min(), temp.max())
    kde = scipy.stats.gaussian_kde(temp)


    plt.plot(z, kde(z), label='length_scale %d'%i)

plt.semilogx()    
plt.legend()

In [ ]:
temp = ystd*2*tsamp['f'][:,1]

# plt.plot(x, (2*temp).mean(0),)
# plt.fill_between(x[:,0], 
#                  (2*temp).mean(0)-2*(2*temp).std(0),
#                  (2*temp).mean(0)+2*(2*temp).std(0),alpha=.1)

plt.plot(xraw, (temp).mean(0), label='$M_2$')
plt.fill_between(xraw[:,0], 
                 (temp).mean(0)-2*(temp).std(0),
                 (temp).mean(0)+2*(temp).std(0),alpha=.3)

temp = ystd*2*nsamp['f'][:,1]

plt.plot(xraw, temp.mean(0), label='$M_0$')
plt.fill_between(xraw[:,0], 
                 temp.mean(0)-2*temp.std(0),
                 temp.mean(0)+2*temp.std(0),alpha=.3)

# plt.plot(xraw, ds.data.loc[:,ds.meta['mM PQ'] != '0.0'].mean(1) - ds.data.loc[:,ds.meta['mM PQ'] == '0.0'].mean(1));

plt.plot([xraw.min(), xraw.max()], [0, 0], lw=3, c='k')

plt.legend(fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel('log(OD)', fontsize=16)
plt.xlabel('time (h)', fontsize=16)

plt.plot()

plt.savefig('figures/ura3_0.083mM-PQ_f1_m02-stan.pdf', bbox_inches='tight')

In [ ]:
plt.plot(ds.data.loc[:,ds.meta['mM PQ'] != '0.0'].mean(1) - ds.data.loc[:,ds.meta['mM PQ'] == '0.0'].mean(1));

In [ ]:
temp = ystd*tsamp['f'][:,0]

# plt.plot(x, (2*temp).mean(0),)
# plt.fill_between(x[:,0], 
#                  (2*temp).mean(0)-2*(2*temp).std(0),
#                  (2*temp).mean(0)+2*(2*temp).std(0),alpha=.1)

plt.plot(xraw, (temp).mean(0), label='$M_2$')
plt.fill_between(xraw[:,0], 
                 (temp).mean(0)-2*(temp).std(0),
                 (temp).mean(0)+2*(temp).std(0),alpha=.3)

temp = ystd*nsamp['f'][:,0]

plt.plot(xraw, temp.mean(0), label='$M_0$')
plt.fill_between(xraw[:,0], 
                 temp.mean(0)-2*temp.std(0),
                 temp.mean(0)+2*temp.std(0),alpha=.3)

plt.plot([xraw.min(), xraw.max()], [0, 0], lw=3, c='k')

plt.legend(fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel('log(OD)', fontsize=16)
plt.xlabel('time (h)', fontsize=16)

# plt.savefig('figures/ura3_0.083mM-PQ_f0_m02-stan.pdf', bbox_inches='tight')